<a href="https://colab.research.google.com/github/Mushtatoes/proj/blob/master/Pokemon_Type_1_to_Highest_Base_Stat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pokemon - Base Stats and Types
**Description:** Is there a correlation between a pokemon's primary type and its highest stat? 


**Goal:** Find a pattern in the minds of pokemon designer and learn more about pandas and matplotlib.
<center>
<img src='https://cdn.clipart.email/cb04f2d0395acc25e2abe9da2b289a5a_pikachu-transparent-background-png-mart_1254-1254.png' height='300'> 
</center>

For example, rock pokemon might have defense as their highest stat. Or maybe Psychic pokemon might have generally high special attack. 

Are there any outlier pokemon that may be a bit different from others of the same type?




In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import re

## 1. Import all unique standard Pokemon:


>- No mega-evolutions, alternate forms, etc.
- Pulling from PokeAPI
- Manually parse and add data into Dataframe without using read_json
- At time of writing, there are currently 807 unique Pokemon in the API (Up to Gen 7)




In [0]:
pokemon_json = requests.get('https://pokeapi.co/api/v2/pokemon?&limit=5000').json()
# print(pokemon_json)

#### 1a. Filter out non-standard Pokemon by id:
>- Create a list of urls that can be used to pull data on each Pokemon.
- PokeAPI uses id <= 10000 to identify standard Pokemon and their traits.
- Use regular expressions to isolate the id from the url and compare.

In [0]:
name_url_raw = pokemon_json['results']

# Filter out non-standard Pokemon using regex on url:
def pokemon_filter(pokemon_object):
  pokemon_search = re.search('^https://pokeapi\.co/api/v2/pokemon/(\d+)/$', pokemon_object['url'])
  if not pokemon_search:
    return False
  return int(pokemon_search.group(1)) <= 10000

name_url = list(filter(pokemon_filter, name_url_raw))

pokemon_urls = [x['url'] for x in name_url]


#### 1b. Map the required data:
>- Each Pokemon's base stats can only be found on their own page, which means a request for each Pokemon.
- Storing into a flat file or SQL table would be preferable if the data is  required for reuse in the future.

In [4]:
# Return each pokemon's name, type1, and base stats:
def pokemon_map(url):
  pokemon = requests.get(url).json()
  data = {'name': pokemon['name']
          ,'type1': ''.join([t['type']['name'] for t in pokemon['types'] if t['slot'] == 1])
          ,'stats': {s['stat']['name']: s['base_stat']for s in pokemon['stats']}}
  return data
  
pokemon_data = list(map(pokemon_map, ['https://pokeapi.co/api/v2/pokemon/1','https://pokeapi.co/api/v2/pokemon/2']))

print(pokemon_data)

[{'name': 'bulbasaur', 'type1': 'grass', 'stats': {'speed': 45, 'special-defense': 65, 'special-attack': 65, 'defense': 49, 'attack': 49, 'hp': 45}}, {'name': 'ivysaur', 'type1': 'grass', 'stats': {'speed': 60, 'special-defense': 80, 'special-attack': 80, 'defense': 63, 'attack': 62, 'hp': 60}}]


#### 1c. Push the data into a pandas DataFrame:
> - Names and types can be extracted into separate lists.
- Each pokemon has 6 base stats:
  - hp
  - attack
  - special-attack
  - defense
  - special-defense
  - speed
- These will have to each be placed into separate lists to push into the DataFrame.

In [5]:
# Format data into dictionaries for DataFrame:
# {Header: [data]}

def pivot_data(category, subcategory = None):
  if not subcategory:
    return [pokemon[category] for pokemon in pokemon_data]
  else:
    return [pokemon[category][subcategory] for pokemon in pokemon_data]

pokemon_name = pivot_data('name')
pokemon_type1 = pivot_data('type1')
pokemon_spd = pivot_data('stats', 'speed')
pokemon_spdef = pivot_data('stats', 'special-defense')
pokemon_spatk = pivot_data('stats', 'special-attack')
pokemon_def = pivot_data('stats', 'defense')
pokemon_atk = pivot_data('stats', 'attack')
pokemon_hp = pivot_data('stats', 'hp')

pokemon_df = pd.DataFrame({'name': pokemon_name
                           ,'type1': pokemon_type1
                           ,'speed': pokemon_spd
                           ,'special-defense': pokemon_spdef
                           ,'special-attack': pokemon_atk
                           ,'defense': pokemon_def
                           ,'attack': pokemon_atk
                           ,'hp': pokemon_hp})


pd.set_option('display.max_columns',8)

print(pokemon_df)

        name  type1  speed  special-defense  special-attack  defense  attack  \
0  bulbasaur  grass     45               65              49       49      49   
1    ivysaur  grass     60               80              62       63      62   

   hp  
0  45  
1  60  
